In [1]:
import sys
sys.path.append('/home/samer/projects/fuzzy_sql/src') #This will enable reading the modules
from pathlib import Path
import os
import json

from fuzzy_sql.rnd_query import RND_QUERY
from fuzzy_sql.fuzzy_sql import *

In [2]:
#set directories
root_dir=Path('/home/samer/projects/fuzzy_sql')
metadata_dir=os.path.join(root_dir,'data/sdgd/processed/metadata')
db_path=os.path.join(root_dir,'db/sdgd.db')

In [3]:
#define input tables and metadata

tbl_names_lst=["C1"]

metadata_lst=[]
for tbl_name in tbl_names_lst:
    with open(os.path.join(metadata_dir,tbl_name+'.json'),'r') as f:
        metadata_lst.append(json.load(f))

In [4]:
#connect to db
conn = sqlite3.connect(db_path)
self=RND_QUERY(conn, tbl_names_lst, metadata_lst)

Metadata for table C1 is valid.
Parameter input is valid.


# Generate Random  queries 

In [6]:
#Single Aggregate
single_expr,groupby_lst,from_tbl, join_tbl_lst, agg_fntn_terms=self.compile_agg_expr() 
rnd_query=self.make_single_agg_query(single_expr,groupby_lst,from_tbl, join_tbl_lst, agg_fntn_terms)


n_rows=rnd_query['query_desc']['n_rows']
n_cols=rnd_query['query_desc']['n_cols']

print(f"no_of_rows={n_rows}\nno_of_cols={n_cols}\nproduct={n_rows*n_cols/1000}")
print("\n")
print(rnd_query['query_desc']['sql'])

no_of_rows=137
no_of_cols=5
product=0.685


SELECT sex, race, occupation, COUNT(*), SUM(hours_per_week)  FROM C1  GROUP BY sex, race, occupation


In [7]:
rnd_query['query_desc']

{'type': 'single_agg',
 'agg_fntn': ('SUM', 'hours_per_week'),
 'grpby_vars': ['sex', 'race', 'occupation'],
 'from_tbl_name': 'C1',
 'join_tbl_name_lst': [],
 'sql': 'SELECT sex, race, occupation, COUNT(*), SUM(hours_per_week)  FROM C1  GROUP BY sex, race, occupation',
 'n_rows': 137,
 'n_cols': 5}

In [8]:
# Twin Aggregate 

syn_tbl_name_lst=['C1_syn_default_1']
real_expr,real_groupby_lst,real_from_tbl, real_join_tbl_lst,agg_fntn_terms=self.compile_agg_expr()
rnd_query=self.make_twin_agg_query(syn_tbl_name_lst,real_expr,real_groupby_lst,real_from_tbl, real_join_tbl_lst,agg_fntn_terms)

n_rows_real=rnd_query['query_desc']['n_rows_real']
n_cols_real=rnd_query['query_desc']['n_cols_real']
n_rows_syn=rnd_query['query_desc']['n_rows_syn']
n_cols_syn=rnd_query['query_desc']['n_cols_syn']

print(rnd_query['query_desc']['sql_real'])
print(f"REAL:\nno_of_rows={n_rows_real}\nno_of_cols={n_cols_real}\nproduct={n_rows_real*n_cols_real/1000}\n\n")

print(rnd_query['query_desc']['sql_syn'])
print(f"SYN:\nno_of_rows={n_rows_syn}\nno_of_cols={n_cols_syn}\nproduct={n_rows_syn*n_cols_syn/1000}\n\n")



TypeError: make_twin_agg_query() missing 5 required positional arguments: 'real_expr', 'real_groupby_lst', 'real_from_tbl', 'real_join_tbl_lst', and 'agg_fntn_terms'

In [ ]:
rnd_query['query_desc']

In [ ]:
#Single Filter

# self.no_groupby_vars=2
# self.no_where_vars=2
# self.max_in_terms=2

rnd_query=self.make_single_fltr_query()


n_rows=rnd_query['query_desc']['n_rows']
n_cols=rnd_query['query_desc']['n_cols']

print(f"no_of_rows={n_rows}\nno_of_cols={n_cols}\nproduct={n_rows*n_cols/1000}")
print("\n")
print(rnd_query['query_desc']['sql'])

In [ ]:
rnd_query['query_desc']

In [ ]:
# Twin Filter 


syn_tbl_name_lst=['C1_syn_default_1']
rnd_query=self.make_twin_fltr_query(syn_tbl_name_lst)

n_rows_real=rnd_query['query_desc']['n_rows_real']
n_cols_real=rnd_query['query_desc']['n_cols_real']
n_rows_syn=rnd_query['query_desc']['n_rows_syn']
n_cols_syn=rnd_query['query_desc']['n_cols_syn']

print(f"REAL:\nno_of_rows={n_rows_real}\nno_of_cols={n_cols_real}\nproduct={n_rows_real*n_cols_real/1000}\n\n")
print(f"SYN:\nno_of_rows={n_rows_syn}\nno_of_cols={n_cols_syn}\nproduct={n_rows_syn*n_cols_syn/1000}\n\n")
print(rnd_query['query_desc']['sql_real'])
print("\n")
print(rnd_query['query_desc']['sql_syn'])

In [ ]:
rnd_query['query_desc']

In [ ]:
#Single Aggregate-Filter

rnd_query=self.make_single_aggfltr_query()


n_rows=rnd_query['query_desc']['n_rows']
n_cols=rnd_query['query_desc']['n_cols']

print(f"no_of_rows={n_rows}\nno_of_cols={n_cols}\nproduct={n_rows*n_cols/1000}")
print("\n")
print(rnd_query['query_desc']['sql'])

In [ ]:
# Twin Aggregate-Filter 

syn_tbl_name_lst=['C1_syn_default_1']
rnd_query=self.make_twin_aggfltr_query(syn_tbl_name_lst)

n_rows_real=rnd_query['query_desc']['n_rows_real']
n_cols_real=rnd_query['query_desc']['n_cols_real']
n_rows_syn=rnd_query['query_desc']['n_rows_syn']
n_cols_syn=rnd_query['query_desc']['n_cols_syn']

print(rnd_query['query_desc']['sql_real'])
print(f"REAL:\nno_of_rows={n_rows_real}\nno_of_cols={n_cols_real}\nproduct={n_rows_real*n_cols_real/1000}\n\n")

print(rnd_query['query_desc']['sql_syn'])
print(f"SYN:\nno_of_rows={n_rows_syn}\nno_of_cols={n_cols_syn}\nproduct={n_rows_syn*n_cols_syn/1000}\n\n")

In [ ]:
matched_query=self._match_twin_query(rnd_query)
scored_query=self.calc_dist_scores(matched_query)

In [ ]:
scored_query['query_desc']

In [ ]:
scored_query['query_hlngr_score']

In [ ]:
scored_query['query_ecldn_score']